First Attempt at using transfer learning with the VGG16 Model: Without Data Augmentation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Load Train and Test Data


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485], std=[0.229])  # Normalize for grayscale X-rays
])

# Load dataset (assumes dataset is in a directory with subfolders for each class)
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/datasets/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/datasets/test', transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers= 4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class to index mapping
print(train_dataset.class_to_idx)  # {'a_normal_xrays': 0, 'bacterial pneumonia': 1, 'viral pneumonia': 2}

2. Load Pretrained VGG16 model

In [ ]:
import torch.nn as nn
import torchvision.models as models


# Load a pretrained VGG16 model
model = models.vgg16(pretrained=True)

# Modify the classifier (for our specific number of classes, e.g., 2 classes for binary classification)
model.classifier[6] = nn.Linear(4096, 3)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

3. First Round of Training: 10 epochs

In [ ]:
import torch.optim as optim

# Freeze all layers except the classifier
for param in model.features.parameters():
    param.requires_grad = False

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer (you can use Adam or SGD)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 10  # Adjust the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        # Move images and labels to the device (GPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

print('Training finished.')

torch.save(model.state_dict(), 'VGG16_on_xray_weights.pth')





Epoch [1/10], Loss: 0.5405989069880152
Epoch [2/10], Loss: 0.3995693463123649
Epoch [3/10], Loss: 0.34103925370731236
Epoch [4/10], Loss: 0.2710988822798788
Epoch [5/10], Loss: 0.19787369323380155
Epoch [6/10], Loss: 0.14987668337906065
Epoch [7/10], Loss: 0.11559754992303856
Epoch [8/10], Loss: 0.09030536668192297
Epoch [9/10], Loss: 0.07235099921900025
Epoch [10/10], Loss: 0.05164470511115087
Training finished.


4. Evaluate Accuracy

In [ ]:
import torch.nn.functional as F

# Function to evaluate the model on the test dataset
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode (no gradient computation)
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in test_loader:
            # Move images and labels to the device (GPU)
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Example usage after training
evaluate_model(model, test_loader)

Test Accuracy: 66.83%


5. Attempt to Improve Accuracy by:
    - training an additional 20 epochs
    - using a learning rate scheduler

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Load the saved model state to resume training
model.load_state_dict(torch.load('VGG16_on_xray_weights.pth', weights_only = True))

# Freeze all layers except the classifier (if not already done)
for param in model.features.parameters():
    param.requires_grad = False

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Continue training for another 20 epochs
num_epochs = 20  # Adjust the number of additional epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        # Move images and labels to the device (GPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss}')

    # Step the scheduler after each epoch, based on the average loss
    scheduler.step(avg_loss)

print('Additional training finished.')

# Save the updated model state after the additional 20 epochs
torch.save(model.state_dict(), 'VGG16_on_xray_weights_updated.pth')

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [1/20], Loss: 0.06268801990799332
Epoch [2/20], Loss: 0.0755457083006858
Epoch [3/20], Loss: 0.05614433191828087
Epoch [4/20], Loss: 0.054475882787423845
Epoch [5/20], Loss: 0.028324159564474666
Epoch [6/20], Loss: 0.04040302057747666
Epoch [7/20], Loss: 0.033309879123921214
Epoch [8/20], Loss: 0.049224281979334095
Epoch [9/20], Loss: 0.03608592369168862
Epoch [10/20], Loss: 0.020256091257685117
Epoch [11/20], Loss: 0.005477504802788672
Epoch [12/20], Loss: 0.006953000420253375
Epoch [13/20], Loss: 0.00325476971110404
Epoch [14/20], Loss: 0.0024957552925110454
Epoch [15/20], Loss: 0.0023376578120333073
Epoch [16/20], Loss: 0.0025249634888462598
Epoch [17/20], Loss: 0.002905294106533891
Epoch [18/20], Loss: 0.0019604945106901006
Epoch [19/20], Loss: 0.0011480812233954577
Epoch [20/20], Loss: 0.0009419707023567109
Additional training finished.


6. Evaluate Model Once More

In [ ]:
#Evaluate model once more
evaluate_model(model, test_loader)

Test Accuracy: 70.51%


Since the learning rate dropped significantly, without seeing an equivalent increase in accuracy, I stopped training here and looked into other methods to improve accuracy. Data Augmentation.